import library

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

load data

In [ ]:
train_df = pd.read_csv(r"D:\Github\Data_Mining_113-2\Data_Mining_Final\dataset\train_data.csv")
train_labels_df = pd.read_csv(r"D:\Github\Data_Mining_113-2\Data_Mining_Final\dataset\train_label.csv")
test_df = pd.read_csv(r"D:\Github\Data_Mining_113-2\Data_Mining_Final\dataset\test_data.csv")

train_ids = train_df.iloc[:, 0]
test_ids = test_df.iloc[:, 0]
X_train = train_df.iloc[:, 1:]
y_train = train_labels_df.iloc[:, 1]
X_test = test_df.iloc[:, 1:]

KNN

In [ ]:
test_result={"id":[],"label":[],"avgdis":[]}
k=5
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

results = []
avg_dis = []
for x in X_test:
    distances = [euclidean_distance(x, x_train) for x_train in X_train]
    k_indices = np.argsort(distances)[:k]
    k_nearest = [(y_train[i], distances[i]) for i in k_indices]
    labels = [label for label, _ in k_nearest]
    most_common = Counter(labels).most_common(1)[0][0]
    results.append(most_common)
    avg_dis.append(np.mean([distance for _, distance in k_nearest]))
df = pd.DataFrame({"id": test_ids, "most_possible_label": results, "avg_dis": avg_dis})

Get each lable in train data that nearlist distance with neighbor

In [70]:
train_avg_dis = {}
for x in range(len(X_train)):
    distances = [euclidean_distance(X_train[x], x_train) for x_train in X_train]
    k_indices = np.argsort(distances)[:k]
    if(train_avg_dis.get(y_train[x])==None): train_avg_dis[y_train[x]] = 0
    train_avg_dis[y_train[x]] += np.mean([distances[i] for i in k_indices])
for key in train_avg_dis:
    train_avg_dis[key] = train_avg_dis[key] / Counter(y_train)[key]
print(train_avg_dis)

{'KIRC': np.float64(120.32907067531288), 'BRCA': np.float64(136.67754293848077), 'LUAD': np.float64(142.04360656843)}


Get known sample & unknown sample

In [71]:
known_sample_id = []
known_sample_label = []
unknown_sample_id = []
for i in range(len(df)):
    if(df.iloc[i, 2] <= train_avg_dis[df.iloc[i, 1]]* 1.5):
        known_sample_id.append(df.iloc[i, 0])
        known_sample_label.append(df.iloc[i, 1])
    else:
        unknown_sample_id.append(df.iloc[i, 0])

known_sample = pd.DataFrame({"id": known_sample_id, "label": known_sample_label})
known_sample.to_csv(r"D:\Github\Data_Mining_113-2\Data_Mining_Final\classification\result\KNN_known.csv", index=False)    
unknown_sample=test_df[~test_df["id"].isin(known_sample_id)]
pd.DataFrame(unknown_sample).to_csv(r"D:\Github\Data_Mining_113-2\Data_Mining_Final\classification\result\KNN_unknown.csv", index=False)

print("number of known sample:", len(known_sample_id),"\nnumber of unknown sample:",  len(unknown_sample_id))

number of known sample: 110 
number of unknown sample: 222
